In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [5]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
tasks+=1
pd.read_sql(
    '''
    SELECT
        sum(case when CRIM is NULL then 1 else 0 end) as crim_null,
        sum(case when ZN is NULL then 1 else 0 end) as zn_null,
        sum(case when INDUS is NULL then 1 else 0 end) as indus_null,
        sum(case when CHAS is NULL then 1 else 0 end) as chas_null,
        sum(case when NOX is NULL then 1 else 0 end) as nox_null
        
    FROM boston
    ''',
    con,
)

,crim_null,zn_null,indus_null,chas_null,nox_null
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [10]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
    COUNT(DISTINCT CRIM) as CRIM,
    COUNT(DISTINCT ZN) as ZN, 
    COUNT(DISTINCT INDUS) as INDUS,
    COUNT(DISTINCT CHAS) as CHAS,
    COUNT(DISTINCT NOX) as NOX
    FROM boston
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [11]:
tasks+=1
pd.read_sql(
    '''
WITH
all_data as
(
    SELECT 'crim' as key, crim as value 
    FROM boston
    
    UNION ALL
    SELECT 'zn' as key, zn as value 
    FROM boston
    
    UNION ALL
    SELECT 'chas' as key, chas as value 
    FROM boston
    
    UNION ALL
    SELECT 'indus' as key, indus as value 
    FROM boston
    
    UNION ALL
    SELECT 'nox' as key, nox as value 
    FROM boston
),
stat_table as
(
    SELECT key, value,
           row_number() over(partition by key order by value) as rn,
           min(value) over(partition by key) min_value,
           count(value) over(partition by key) as cnt
    FROM all_data
)
    

SELECT key FROM (
                    SELECT key, value as md_value, min_value
                    FROM stat_table
                    WHERE rn = cnt/2
                )
WHERE md_value=min_value
    ''',
    con,
)

,key
0,chas
1,zn


##### Больше половины участков не граничат с рекой и имеют площадь менее 25000 кв. футов.

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [12]:
tasks+=1
pd.read_sql(
    '''
    SELECT 25 as "Количество домов", AVG(ARM) as 'Среднее кол-во комнат',
    
    (SELECT AVG(RM) OVER(partition by MEDV order by MEDV) as AVGRM_MAXPRICE
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1) as 'Среднее кол-во комнат в самых дорогих',
    
    AVG(ARM-(
    SELECT AVG(RM) OVER(partition by MEDV order by MEDV) as AVGRM_MAXPRICE
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1)) as Разница
    FROM (SELECT RM as ARM FROM boston ORDER BY MEDV LIMIT 25)
    
    UNION ALL
    SELECT 50 as "Количество домов", AVG(ARM) as 'Среднее кол-во комнат',
    
    (SELECT AVG(RM) OVER(partition by MEDV order by MEDV) as AVGRM_MAXPRICE
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1) as 'Среднее кол-во комнат в самых дорогих',
    
    AVG(ARM-(
    SELECT AVG(RM) OVER(partition by MEDV order by MEDV) as AVGRM_MAXPRICE
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1)) as Разница
    FROM (SELECT RM as ARM FROM boston ORDER BY MEDV LIMIT 50)
    
    UNION ALL
    SELECT 100 as "Количество домов", AVG(ARM) as 'Среднее кол-во комнат',
    
    (SELECT AVG(RM) OVER(partition by MEDV order by MEDV) as AVGRM_MAXPRICE
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1) as 'Среднее кол-во комнат в самых дорогих',
    
    AVG(ARM-(
    SELECT AVG(RM) OVER(partition by MEDV order by MEDV) as AVGRM_MAXPRICE
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1)) as Разница
    FROM (SELECT RM as ARM FROM boston ORDER BY MEDV LIMIT 100)
    
    UNION ALL
    SELECT 200 as "Количество домов", AVG(ARM) as 'Среднее кол-во комнат',
    
    (SELECT AVG(RM) OVER(partition by MEDV order by MEDV) as AVGRM_MAXPRICE
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1) as 'Среднее кол-во комнат в самых дорогих',
    
    AVG(ARM-(
    SELECT AVG(RM) OVER(partition by MEDV order by MEDV) as AVGRM_MAXPRICE
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1)) as Разница
    FROM (SELECT RM as ARM FROM boston ORDER BY MEDV LIMIT 200)
    
    UNION ALL
    SELECT 300 as "Количество домов", AVG(ARM) as 'Среднее кол-во комнат',
    
    (SELECT AVG(RM) OVER(partition by MEDV order by MEDV) as AVGRM_MAXPRICE
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1) as 'Среднее кол-во комнат в самых дорогих',
    
    AVG(ARM-(
    SELECT AVG(RM) OVER(partition by MEDV order by MEDV) as AVGRM_MAXPRICE
    FROM boston
    ORDER BY MEDV DESC
    LIMIT 1)) as Разница
    FROM (SELECT RM as ARM FROM boston ORDER BY MEDV LIMIT 300)

    ''',
    con,
)

,Количество домов,Среднее кол-во комнат,Среднее кол-во комнат в самых дорогих,Разница
0,25,5.747840,7.484,-1.736160
1,50,5.753240,7.484,-1.730760
2,100,5.887120,7.484,-1.596880
3,200,5.911705,7.484,-1.572295
4,300,5.972227,7.484,-1.511773


#####  Опираясь на данные можно сделать вывод о том, что кол-во комнат влияет на стоимость жилой недвижимости (больше комнат - дороже недвижимость)

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT LSTAT, MEDV, Rank_LSTAT FROM(
    SELECT rank() OVER(ORDER BY LSTAT) as Rank_LSTAT, *
    FROM boston)
    WHERE MEDV = (SELECT MAX(MEDV) FROM boston)
    ''',
    con,
)

,LSTAT,MEDV,Rank_LSTAT
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


##### Процент (LSTAT) более низкого статуса населения идет на увеличение в зависимости от стоимость недвижимости. 

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [14]:
tasks+=1
pd.read_sql(
    """
    SELECT AVG(MEDV) as 'Средняя стоимость домов', 'Граничит' as "Граничит ли с рекой"
    FROM boston
    WHERE CHAS = 1
    
    UNION ALL
    SELECT AVG(MEDV), 'Не граничит'
    FROM boston
    WHERE CHAS = 0
    """,
    con,
)

,Средняя стоимость домов,Граничит ли с рекой
0,28.440000,Граничит
1,22.093843,Не граничит


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [15]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM(
    SELECT 'CRIM' as 'Name_col', (SELECT AVG(CRIM) FROM boston WHERE CHAS = 0) as 'CHAS_0', 
    (SELECT AVG(CRIM) FROM boston WHERE CHAS = 1) as 'CHAS_1'
    
    UNION ALL
    SELECT 'ZN' as 'Name_col', (SELECT AVG(ZN) FROM boston WHERE CHAS = 0) as 'CHAS_0', 
    (SELECT AVG(ZN) FROM boston WHERE CHAS = 1) as 'CHAS_1'
    
    UNION ALL
    SELECT 'INDUS' as 'Name_col', (SELECT AVG(INDUS) FROM boston WHERE CHAS = 0) as 'CHAS_0', 
    (SELECT AVG(INDUS) FROM boston WHERE CHAS = 1) as 'CHAS_1'
    
    UNION ALL
    SELECT 'NOX' as 'Name_col', (SELECT AVG(NOX) FROM boston WHERE CHAS = 0) as 'CHAS_0', 
    (SELECT AVG(NOX) FROM boston WHERE CHAS = 1) as 'CHAS_1'
    )
    WHERE CHAS_0 < CHAS_1
    ''',
    con,
)

,Name_col,CHAS_0,CHAS_1
0,INDUS,11.019193,12.719143
1,NOX,0.551817,0.593426


##### На участках у реки выше показатель доли акров неторгового бизнеса а также концентрации оксидов азота. 

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [13]:
tasks+=1
pd.read_sql(
    '''
    WITH data as(    
    SELECT *, ntile(10) over(order by nox) as nox_prc,
              ntile(10) over(order by indus) as indus_prc
    FROM boston)
    
    
SELECT  t1.percentile, nox_value, indus_value
FROM 
       (SELECT distinct nox_prc*10 as percentile, max(nox) over(partition by nox_prc) nox_value
        FROM data ) t1 JOIN
        (SELECT distinct indus_prc*10 as percentile, max(indus) over(partition by indus_prc) indus_value
        FROM data) t2
ON t1.percentile = t2.percentile
    ''',
    con,
)

,percentile,nox_value,indus_value
0,10,0.426,2.89
1,20,0.442,4.39
2,30,0.472,5.96
3,40,0.507,7.38
4,50,0.538,9.69
5,60,0.580,13.89
6,70,0.609,18.10
7,80,0.671,18.10
8,90,0.713,19.58
9,100,0.871,27.74


In [17]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
